In [ ]:
#from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle


In [6]:
prepath = "../street_view_house_numbers/"
pickle_file = prepath+'SVHN.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  #train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  #print('Training set', train_dataset.shape, train_labels.shape)
  #print('Validation set', valid_dataset.shape, valid_labels.shape)
  #print('Test set', test_dataset.shape, test_labels.shape)

In [7]:
pickle_file = prepath+'SVHN1.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset1 = save['train_dataset1']
  del save  # hint to help gc free up memory

pickle_file = prepath+'SVHN2.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset2 = save['train_dataset2']
  del save  # hint to help gc free up memory

pickle_file = prepath+'SVHN3.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset3 = save['train_dataset3']
  del save  # hint to help gc free up memory

In [8]:
train_dataset = np.concatenate((train_dataset1, train_dataset2, train_dataset3), axis=0)
del train_dataset1, train_dataset2, train_dataset3
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (598388, 32, 32), (598388,))
('Validation set', (6000, 32, 32), (6000,))
('Test set', (26032, 32, 32), (26032,))


In [10]:
image_size = 32
num_labels = 10
num_channels = 1 # grayscale

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = labels.astype(np.int32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (598388, 32, 32, 1), (598388,))
('Validation set', (6000, 32, 32, 1), (6000,))
('Test set', (26032, 32, 32, 1), (26032,))


In [26]:
def dense_to_one_hot(labels_dense, num_classes=10):
  """Convert class labels from scalars to one-hot vectors."""
  num_labels = labels_dense.shape[0]
  index_offset = np.arange(num_labels) * num_classes
  labels_one_hot = np.zeros((num_labels, num_classes))
  labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
  return labels_one_hot

In [31]:
train_labels_1hot = dense_to_one_hot(train_labels)
valid_labels_1hot = dense_to_one_hot(valid_labels)
test_labels_1hot = dense_to_one_hot(test_labels)

In [88]:
def grab_next_train_batch(batch):
  offset = (batch * batch_size) % (train_labels.shape[0])
  batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
  batch_labels = train_labels_1hot[offset:(offset + batch_size)]
#   print batch, offset
  return batch_data, batch_labels


def grab_next_test_batch(batch):
  random_indices = np.arange(len(test_dataset))
  np.random.shuffle(random_indices)
  indices = random_indices[:5000]
  
  batch_data = test_dataset[indices, :, :, :]
  batch_labels = test_labels_1hot[indices]
  return batch_data, batch_labels

In [ ]:
import numpy as np
import os
from datetime import datetime
from time import time
import tensorflow as tf


# Data information
input_channels = train_dataset.shape[3]
image_size = 32
n_classes = 10 # total classes (0-9 digits)

##########################################
##                                      ##
##            Helper Wrappers           ##
##                                      ##
##########################################

# Default stdev for weights and biases
init_bias = 0.1
init_std = 0.1
# Default decay, if non-zero
init_decay = 0.0#0.004


# We can't initialize these variables to 0 - the network will get stuck.
def weight_variable(shape, stddev=init_std):
  """Create a weight variable with appropriate initialization."""
  initial = tf.truncated_normal(shape, stddev)
  return tf.Variable(initial)


def bias_variable(shape, default_bias=init_bias):
  """Create a bias variable with appropriate initialization."""
  initial = tf.constant(default_bias, shape=shape)
  return tf.Variable(initial)


def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)


def convlayer(layer_name, input_tensor, receptive_field, channels_in, channels_out,
              padding='SAME', stride=1, act=tf.nn.relu, decay=0,
              pool=False, pooler=tf.nn.max_pool, pool_size=2, pool_stride=2, pool_padding='SAME',
              batch_norm=True, training=True):
  """General purpose convolutional layer, followed by pooling

  It does a matrix convolution, bias add, and then uses relu by default to nonlinearize.
  Then it pools using max pooling by default.
  It also sets up name scoping so that the resultant graph is easy to read,
  and adds a number of summary ops for TensorBoard.
  """
  # Adding a name scope ensures logical grouping of the layers in the graph.
  with tf.name_scope(layer_name):
    # This Variable will hold the state of the weights for the layer
    with tf.name_scope('weights'):
      weights = weight_variable([receptive_field, receptive_field, channels_in, channels_out])
      variable_summaries(weights)

      if decay > 0:
          weight_decay = tf.multiply(tf.nn.l2_loss(weights), decay, name='weight_decay')
          tf.add_to_collection('losses', weight_decay)

    with tf.name_scope('biases'):
      biases = bias_variable([channels_out])
      # variable_summaries(biases)

    with tf.name_scope('W_conv_x_plus_b'):
      preactivate = tf.nn.conv2d(input_tensor, weights,
                                 strides=[1, stride, stride, 1],
                                 padding=padding) + biases
      tf.summary.histogram('pre_activations', preactivate)

    if batch_norm:
      with tf.name_scope('batchnorm'):
        normed = tf.layers.batch_normalization(preactivate, training=training)
      activations = act(normed, name='activation')
    else:
      activations = act(preactivate, name='activation')
    tf.summary.histogram('activations', activations)

    if pool:
      max_pool = pooler(activations, ksize=[1, pool_size, pool_size, 1],
                      strides=[1, pool_stride, pool_stride, 1],
                      padding=pool_padding)
      tf.summary.histogram('pools', max_pool)
      return max_pool
    else:
      return activations


def nn_layer(layer_name, input_tensor, input_dim, output_dim, act=tf.nn.relu, decay=0):
  """Reusable code for making a normal neural net layer.
  It does a matrix multiply, bias add, and then uses ReLU to nonlinearize.
  It also sets up name scoping so that the resultant graph is easy to read,
  and adds a number of summary ops.
  """
  # Adding a name scope ensures logical grouping of the layers in the graph.
  with tf.name_scope(layer_name):
    # This Variable will hold the state of the weights for the layer
    with tf.name_scope('weights'):
      weights = weight_variable([input_dim, output_dim])

      if decay > 0:
        weight_decay = tf.multiply(tf.nn.l2_loss(weights), decay, name='weight_decay')
        tf.add_to_collection('losses', weight_decay)

      # variable_summaries(weights)
    with tf.name_scope('biases'):
      biases = bias_variable([output_dim])
      # variable_summaries(biases)
    with tf.name_scope('Wx_plus_b'):
      preactivate = tf.matmul(input_tensor, weights) + biases
      tf.summary.histogram('pre_activations', preactivate)
    activations = act(preactivate, name='activation')
    tf.summary.histogram('activations', activations)
    return activations


def flat_dimension(tensor):
  dim = 1 # Compute how many numbers we have, ignoring the batch size
  for d in tensor.get_shape()[1:].as_list():
    dim *= d
  return dim




def run():
  # RESET TF GRAPH, just in case
  tf.reset_default_graph()

  sess = tf.InteractiveSession() ###### May need to change for .py files

  ts = datetime.now().strftime('%Y%m%d_%H%M')
  logs_path = "logs/{}/".format(ts)

  ### Place holders ###
  # Optionally track that place holders are correctly set at test and train tme
  with tf.name_scope('test_train_variables'):
    training = tf.placeholder(tf.bool) # for batch_norm mode
    keep_prob = tf.placeholder(tf.float32) # for drop out
    tf.summary.scalar('training', tf.to_int32(training, name='ToInt32'))
    tf.summary.scalar('dropout_keep_probability', keep_prob)


  with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, shape=[None, image_size, image_size, input_channels], name="x-input")
    y_ = tf.placeholder(tf.float32, shape=[None, n_classes], name="y-input")


  conv1 = convlayer(layer_name='conv1', input_tensor=x, receptive_field=5,
                      channels_in=input_channels, channels_out=32, pool=False, pool_size=2, pool_stride=2,
                      batch_norm=True, training=training)

  conv2 = convlayer(layer_name='conv2', input_tensor=conv1, receptive_field=5,
                    channels_in=32, channels_out=64, pool=False, pool_size=2, pool_stride=2,
                    batch_norm=True, training=training)
  
  conv3 = convlayer(layer_name='conv3', input_tensor=conv2, receptive_field=5,
                  channels_in=64, channels_out=64, pool=False, pool_size=2, pool_stride=2,
                  batch_norm=True, training=training)


  last_conv_layer = conv3

  with tf.name_scope('conv2_flatten'):
    conv_reshaped = tf.reshape(last_conv_layer, [-1, flat_dimension(last_conv_layer)])

  fc1 = nn_layer(layer_name='fc1', input_tensor=conv_reshaped, input_dim=flat_dimension(last_conv_layer), output_dim=1024, decay=init_decay)
  dropped1 = tf.nn.dropout(fc1, keep_prob)
  
  fc2 = nn_layer(layer_name='fc2', input_tensor=dropped1, input_dim=1024, output_dim=512, decay=init_decay)
  dropped2 = tf.nn.dropout(fc2, keep_prob)
  
  last_dense_layer = dropped2

  # Do not apply softmax activation yet! use the identity
  y = nn_layer(layer_name='output', input_tensor=last_dense_layer, input_dim=512, output_dim=n_classes, act=tf.identity)



  # Cross-Entropy Loss
  with tf.name_scope('cross_entropy'):
    diff = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)
    with tf.name_scope('total'):
      cross_entropy = tf.reduce_mean(diff)
      tf.add_to_collection('losses', cross_entropy)
  tf.summary.scalar('cross_entropy', cross_entropy)
  # Total loss (weight decay + cross-entropy)
  total_loss = tf.add_n(tf.get_collection('losses'), name='total_loss')


  with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)


  # Other metrics
  with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
      correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    with tf.name_scope('accuracy'):
      accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  tf.summary.scalar('accuracy', accuracy)

  # Merge all the summaries and write them out
  merged = tf.summary.merge_all()
  train_writer = tf.summary.FileWriter(logs_path + '/train', sess.graph)
  test_writer = tf.summary.FileWriter(logs_path + '/test')

  # Normalize by subtracting per image, per channel means
  def normalize_batch(batch):
    per_img_ch_means = batch.mean(axis=1)
    return batch - per_img_ch_means[:, np.newaxis, :]

  # Different options for running the graph
  def feed_dict(mode, batch):
    """Make a TensorFlow feed_dict: maps data onto Tensor placeholders."""
    if mode == 'Train':
      #batch_x, batch_y = svhn_train.next_batch(batch_size)
      batch_x, batch_y = grab_next_train_batch(batch)
      keep_proba = train_keep_prob
      training_ = True
    elif mode == 'Train_no_drop':
      batch_x, batch_y = grab_next_train_batch(batch)
      keep_proba = 1.0
      training_ = True
    elif mode == 'Test':
      batch_x, batch_y = grab_next_test_batch(batch)
      keep_proba = 1.0
      training_ = False
    batch_x = normalize_batch(batch_x) # Subtract per image mean
    return {x: batch_x, y_: batch_y, keep_prob: keep_proba, training: training_}


  # Might be needed for batch norm
  extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
  tf.global_variables_initializer().run()


  t_start = time()

  # Training loop
  for batch in xrange(total_batches):
    
    if batch % test_every == test_every - 1:
      # Record summaries and accuracy on the *test* set
      summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict('Test', batch))

      test_writer.add_summary(summary, batch)

      print('Test accuracy at batch %s: %s' % (batch, acc))
      # Now proceed to train and produce corresponding training summary too

      summary, _, _ = sess.run([merged, train_step, extra_update_ops], feed_dict=feed_dict('Train_no_drop', batch))
      train_writer.add_summary(summary, batch)

    else:
      sess.run([train_step, extra_update_ops], feed_dict=feed_dict('Train', batch))


  print "\nOptimization Finished!\n"
  t_end = time()
  print (t_end - t_start) / 60.0, " minutes"
  print (t_end - t_start) / 3600.0, " hours"


  pwd = os.getcwd()+"/"
  print("Run the following to start tensorboard server:\n"         "tensorboard --logdir=/{}{}".format(pwd, logs_path))
  train_writer.close()
  test_writer.close()


In [ ]:
##########################################
##                                      ##
##              Parameters              ##
##                                      ##
##########################################

learning_rate = 0.001 # Slightly higher since we are using batch norm
train_keep_prob = 0.975 # Low dropout, in addition to weight decay

# Train batch size
batch_size = 128 # Better at 128 or 256
total_batches = 3 * int(len(train_dataset) / batch_size) # Train on all of the data

# Test frequency / size
test_every = 200 # Record test accuracy every 500 batches (32*500 examples) -- ideally every 100

run()


Test accuracy at batch 199: 0.6486
Test accuracy at batch 399: 0.7516
Test accuracy at batch 599: 0.756
Test accuracy at batch 799: 0.784
